<a href="https://colab.research.google.com/github/JONNY-ME/AIBot-telegram/blob/master/Combined_Anglophone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Module Installation

In [65]:
!pip install catboost -q
!pip install pycountry_convert -q
from google.colab import drive
drive.mount('/content/drive')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires coverage==3.7.1, but you have coverage 6.4.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
coveralls 0.5 requires coverage<3.999,>=3.6, but you have coverage 6.4.1 which is incompatible.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading Modules




In [66]:
import os
import numpy as np
import pandas as pd
import warnings
import pycountry_convert as pc
from catboost import CatBoostClassifier
from itertools import combinations
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



path = "/content/drive/MyDrive/Anglophpne/"
seed = 21
le = LabelEncoder()
os.chdir(path)
warnings.filterwarnings('ignore')

## Utility Functions

In [101]:
def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name


def add_continent(df):
  continents_map = {}
  for country in df.country.unique():
      if country == 'UNITED STATES OF AMERICA':
          continents_map[country] = country_to_continent('United States of America')
      elif country == 'UAE':
          continents_map[country] = country_to_continent('United Arab Emirates')
      elif country == 'KOREA':
          continents_map[country] = country_to_continent('South Korea')
      elif country == 'DRC':
          continents_map[country] = country_to_continent('Democratic Republic of the Congo')
      elif country == 'SAUD ARABIA':
          continents_map[country] = country_to_continent('Saudi Arabia')
      elif country == 'TRINIDAD TOBACCO':
          continents_map[country] = country_to_continent('Trinidad and Tobago')
      elif country == 'BOSNIA':
          continents_map[country] = country_to_continent('Bosnia and Herzegovina')
      elif country == 'SCOTLAND':
          continents_map[country] =  'Europe'
      elif country == 'UKRAIN':
          continents_map[country] = 'Europe'
      elif country == 'SOMALI':
          continents_map[country] = 'Africa'
      else:
          continents_map[country] = country_to_continent(country.title())
  df['Continent'] = df.country.apply(continents_map.get)

## Loading Data

In [102]:
Train = pd.read_csv("Train.csv")
Test = pd.read_csv("Test.csv")

## Preprocessing

In [183]:
Train["is_train"] = 1
Test["is_train"] = 0

train_test = pd.concat([Train, Test])
train_test.shape

(24675, 22)

In [184]:
wrg_crt = {'SWIZERLAND':'SWITZERLAND', 'MORROCO':'MOROCCO', 'MALT':'MALTA', 'COSTARICA':'COSTA RICA', 'DJIBOUT':'DJIBOUTI',
          'BURGARIA':'BULGARIA', 'PHILIPINES':'PHILIPPINES', 'COMORO':'COMOROS', 'MONECASQUE':'MONACO', 'ECUADO':'ECUADOR'
  }
for wrg, crt in wrg_crt.items():
  train_test["country"].loc[train_test.country == wrg] = crt


train_test["main_activity"].loc[train_test.main_activity == "Widife Tourism"] = "Wildlife Tourism"
train_test["total_person"] = train_test["total_female"] + train_test["total_male"]
train_test['person_str'] = train_test['total_male'].astype(str) + train_test['total_female'].astype(str)
train_test['nights_str'] = train_test['night_mainland'].astype(str) + train_test['night_zanzibar'].astype(str)


packages = ["package_transport_int", "package_accomodation", "package_food", "package_transport_tz", 
            "package_sightseeing", "package_guided_tour", "package_insurance"]
for package in packages:
  train_test[package] = (train_test[package] == "Yes").astype(int)

exec(f"train_test['full_package'] = " + " & ".join(map(lambda x:f'train_test["{x}"]', packages)))

add_continent(train_test)


In [185]:
CountryCosts = dict(train_test.groupby('country')['cost_category'].agg(lambda x:(list(x.value_counts().index)+['Normal Cost'])[0]))
ContinentsCosts = dict(train_test.groupby('Continent')['cost_category'].agg(lambda x: x.value_counts().index[0]))
AgesCosts = dict(train_test.groupby('age_group')['cost_category'].agg(lambda x: x.value_counts().index[0]))
PurposeCosts = dict(train_test.groupby('purpose')['cost_category'].agg(lambda x: x.value_counts().index[0]))
ActivityCosts = dict(train_test.groupby('main_activity')['cost_category'].agg(lambda x: x.value_counts().index[0]))
InfoCosts = dict(train_test.groupby('info_source')['cost_category'].agg(lambda x: x.value_counts().index[0]))


train_test['CountryCosts'] = train_test['country'].map(CountryCosts)
train_test['ContinentsCosts'] = train_test['Continent'].map(ContinentsCosts)
train_test['AgesCosts'] = train_test['age_group'].map(AgesCosts)
train_test['PurposeCosts'] = train_test['purpose'].map(PurposeCosts)
train_test['ActivityCosts'] = train_test['main_activity'].map(ActivityCosts)
train_test['InfoCosts'] = train_test['info_source'].map(InfoCosts)

Missing values

In [186]:
bl = train_test.travel_with.isna()
train_test['travel_with'][bl & (train_test.total_person==1)] = 'Alone'
train_test['travel_with'][bl & (train_test.total_male==1) & (train_test.total_female==1)] = 'Spouse'

train_test['travel_with'] = train_test['travel_with'].fillna('Friends/Relatives') 

for per in ['total_person', 'total_female', 'total_male']:
    train_test[per].fillna(0, inplace=True)

In [187]:
cat_cols = ["country", "age_group", "travel_with", "purpose", "main_activity", "info_source", "tour_arrangement", 
            "first_trip_tz", "person_str", "nights_str", "Continent"]
for col in cat_cols:
  train_test[col] = le.fit_transform(train_test[col])


cost_cols = ["CountryCosts", "ContinentsCosts", "AgesCosts", "PurposeCosts", "ActivityCosts", "InfoCosts"]
cost_map = {
    'Lower Cost' : 0,
    'Low Cost' : 1,
    'Normal Cost' : 2,
    'High Cost' : 3,
    'Higher Cost' : 4,
    'Highest Cost' : 5
}

for cols in cost_cols:
    train_test[cols] = train_test[cols].apply(cost_map.get)

In [188]:
train_test

,Tour_ID,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,...,person_str,nights_str,full_package,Continent,CountryCosts,ContinentsCosts,AgesCosts,PurposeCosts,ActivityCosts,InfoCosts
0,tour_id1hffseyw,61,2,3,0.0,2.0,6,0,0,1,...,57,44,0,2,3,2,4,0,2,2
1,tour_idnacd7zag,132,1,5,1.0,1.0,1,9,7,1,...,12,44,0,2,2,2,2,4,4,4
2,tour_id62vz7e71,133,3,5,1.0,1.0,1,8,7,1,...,12,606,0,3,4,4,4,4,4,4
3,tour_idrc76tzix,105,1,6,3.0,1.0,1,0,4,0,...,18,349,0,0,0,0,2,4,2,2
4,tour_idn723m0n9,133,2,0,0.0,1.0,1,8,7,1,...,11,618,0,3,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6164,tour_id2deyfjhq,139,1,0,0.0,1.0,0,9,0,0,...,11,231,0,0,0,0,2,2,4,2
6165,tour_idlenv2rio,37,1,0,0.0,1.0,6,6,0,0,...,11,573,0,0,0,0,2,0,0,2
6166,tour_id7wwqrs0p,22,1,0,0.0,1.0,1,0,0,1,...,11,518,0,3,4,4,2,4,2,2
6167,tour_idx80vbw5a,22,0,0,1.0,0.0,6,9,0,0,...,1,250,0,3,4,4,2,0,4,2


In [189]:
train = train_test[train_test["is_train"] == 1].drop("is_train", axis=1)

In [190]:
cost_map1 = {
    'Lower Cost': 0,
    'Low Cost': 0,
    'Normal Cost': 1,
    'High Cost': 2, 
    'Higher Cost': 2, 
    'Highest Cost': 2,
}
cost_map2 = {
    'Lower Cost' : 0,
    'Low Cost' : 1,
    'Normal Cost': 1,
    'High Cost' : 0,
    'Higher Cost' : 1,
    'Highest Cost' : 2,
}


train["category"] = train["cost_category"].apply(cost_map1.get)
train["spec_category"] = train["cost_category"].apply(cost_map2.get)
train["cost_category"] = train["cost_category"].apply(cost_map.get)

In [191]:
X = train.drop(["Tour_ID", "cost_category", 'category', 'spec_category'], axis=1)
y = train[['category', "cost_category", "spec_category"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [192]:
test = train_test[train_test["is_train"]==0].drop(["cost_category", "is_train"], axis=1)
test.fillna(0, inplace=True)

## Modeling and Prediction

In [193]:

rnf_params = {'max_depth': 16, 'max_features':'log2', 'n_estimators': 1000, 'random_state': seed}
lgb_prams = {"learning_rate":0.009, "n_estimators":1000, 'random_state':seed}
xgb_params = {"eta":0.3, 'n_estimators': 200, 'random_state':seed}
cat_params = {"iterations":1000, "learning_rate":0.01, "verbose":0, "random_state":seed, }

def model():
  models = [
            ('lgb', LGBMClassifier(**lgb_prams)),
            ('rnd', RandomForestClassifier(**rnf_params)),
            ('xgb', XGBClassifier(**xgb_params)),
            ('cat', CatBoostClassifier(**cat_params)),
  ]

  return VotingClassifier(estimators=models, voting='soft', flatten_transform=True)


#### Despatched model

##### Training

In [194]:
main_model = model()

main_model.fit(X_train.values, y_train.category)

pred = main_model.predict_proba(X_test.values)
pred_df = pd.DataFrame(pred, columns=["Low", "Medium", "High"], index=X_test.index)
log_loss(y_test.category, pred)

0.6291967107625088

In [195]:
preds = {}
pred_df['mlower'] = 1
pred_df['mlow'] = 1
pred_df['mnorm'] = 1
pred_df['mhigh'] = 1
pred_df['mhigher'] = 1
pred_df['mhighest'] = 1

cols = [['mlower', 'mlow'], ['mnorm'], ['mhigh', 'mhigher', 'mhighest']]
for cat in [0, 2]:
  inx1 = y_train[y_train.category == cat].index
  xtr = X_train.loc[inx1]
  ytr = y_train.loc[inx1]

  clf = model()
  clf.fit(xtr, ytr.spec_category)
  pred = clf.predict_proba(X_test)
  for i in range(pred.shape[1]):
    pred_df[cols[cat][i]] = pred[:, i]


pred_df


,Low,Medium,High,mlower,mlow,mnorm,mhigh,mhigher,mhighest
7604,0.400042,0.557881,0.042077,0.389534,0.610466,1,0.773674,0.221526,0.004800
3169,0.018576,0.090920,0.890504,0.608550,0.391450,1,0.437842,0.552654,0.009505
17535,0.049895,0.066229,0.883876,0.738842,0.261158,1,0.113915,0.859340,0.026744
7628,0.692710,0.281104,0.026185,0.601317,0.398683,1,0.789587,0.205756,0.004657
9973,0.043922,0.370595,0.585483,0.803466,0.196534,1,0.728171,0.261788,0.010042
...,...,...,...,...,...,...,...,...,...
12674,0.852248,0.135767,0.011985,0.807143,0.192857,1,0.665147,0.332384,0.002469
10066,0.021222,0.053954,0.924825,0.714205,0.285795,1,0.222018,0.605129,0.172853
3099,0.013545,0.049351,0.937104,0.260591,0.739409,1,0.340798,0.654114,0.005088
5999,0.291484,0.647804,0.060712,0.434394,0.565606,1,0.676975,0.320150,0.002875


In [196]:
pred_df['Lower Cost'] = pred_df['Low'] * pred_df['mlower']
pred_df['Low Cost'] = pred_df['Low'] * pred_df['mlow']
pred_df['Normal Cost'] = pred_df['Medium'] * pred_df['mnorm']
pred_df['High Cost'] = pred_df['High'] * pred_df['mhigh']
pred_df['Higher Cost'] = pred_df['High'] * pred_df['mhigher']
pred_df['Highest Cost'] = pred_df['High'] * pred_df['mhighest']


_pred = pred_df[list(cost_map.keys())]
log_loss(y_test.cost_category, _pred)

1.0780715578955498

##### Prediction

In [197]:
main_model.fit(X.values, y.category)
pred = main_model.predict_proba(test.drop("Tour_ID", axis=1).values)
pred_df = pd.DataFrame(pred, columns=["Low", "Medium", "High"], index=test.index)

In [198]:
preds = {}
pred_df['mlower'] = 1
pred_df['mlow'] = 1
pred_df['mnorm'] = 1
pred_df['mhigh'] = 1
pred_df['mhigher'] = 1
pred_df['mhighest'] = 1

cols = [['mlower', 'mlow'], ['mnorm'], ['mhigh', 'mhigher', 'mhighest']]
for cat in [0, 2]:
  inx1 = y[y.category == cat].index
  xtr = X.loc[inx1]
  ytr = y.loc[inx1]


  clf = model()
  clf.fit(xtr, ytr.spec_category)
  pred = clf.predict_proba(test.drop("Tour_ID", axis=1))
  for i in range(pred.shape[1]):
    pred_df[cols[cat][i]] = pred[:, i]


pred_df

,Low,Medium,High,mlower,mlow,mnorm,mhigh,mhigher,mhighest
0,0.118816,0.742550,0.138634,0.364311,0.635689,1,0.747373,0.250485,0.002141
1,0.025070,0.123531,0.851399,0.726932,0.273068,1,0.362946,0.595620,0.041435
2,0.014581,0.114667,0.870751,0.242439,0.757561,1,0.810270,0.183818,0.005911
3,0.561756,0.391044,0.047200,0.295712,0.704288,1,0.617006,0.379956,0.003038
4,0.598328,0.374668,0.027004,0.580728,0.419272,1,0.754657,0.243109,0.002235
...,...,...,...,...,...,...,...,...,...
6164,0.801179,0.180831,0.017991,0.642794,0.357206,1,0.800646,0.196999,0.002355
6165,0.706194,0.248606,0.045200,0.872189,0.127811,1,0.596495,0.398317,0.005188
6166,0.039005,0.356034,0.604961,0.453840,0.546160,1,0.671339,0.321504,0.007157
6167,0.329874,0.562370,0.107756,0.388926,0.611074,1,0.832139,0.166509,0.001352


In [199]:
pred_df['Lower Cost'] = pred_df['Low'] * pred_df['mlower']
pred_df['Low Cost'] = pred_df['Low'] * pred_df['mlow']
pred_df['Normal Cost'] = pred_df['Medium'] * pred_df['mnorm']
pred_df['High Cost'] = pred_df['High'] * pred_df['mhigh']
pred_df['Higher Cost'] = pred_df['High'] * pred_df['mhigher']
pred_df['Highest Cost'] = pred_df['High'] * pred_df['mhighest']


pred_df['Tour_ID'] = test.Tour_ID
sub1 = pred_df[['Tour_ID', 'High Cost', 'Higher Cost', 'Highest Cost', 'Low Cost', 'Lower Cost', 'Normal Cost']]
sub1.head()

,Tour_ID,High Cost,Higher Cost,Highest Cost,Low Cost,Lower Cost,Normal Cost
0,tour_idynufedne,0.103611,0.034726,0.000297,0.075530,0.043286,0.742550
1,tour_id9r3y5moe,0.309012,0.507110,0.035277,0.006846,0.018224,0.123531
2,tour_idf6itml6g,0.705544,0.160060,0.005147,0.011046,0.003535,0.114667
3,tour_id99u4znru,0.029123,0.017934,0.000143,0.395638,0.166118,0.391044
4,tour_idj4i9urbx,0.020379,0.006565,0.000060,0.250862,0.347466,0.374668


In [200]:
sub1.to_csv('/content/mo-feats.csv', index=False)